In [2]:
import re
import time
import pandas as pd
from selenium import webdriver
from pyproj import Proj, Transformer
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
url = f"https://new.land.naver.com/offices?ms=35.154807,126.9011,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get(url)

In [4]:
# 데이터프레임 초기화
columns = [
    '제목',
    '월세',
    '소재지',
    '매물특징',
    '계약/전용면적',
    '해당층/총층',
    '입주가능일',
    '권리금',
    '현재업종',
    '추천업종',
    '준공인가일',
    '매물번호',
    '위도',
    '경도'
]
df = pd.DataFrame(columns=columns)

In [15]:
# 지도 중심 좌표 계산
map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
map_center_x = map_element.size['width'] / 2
map_center_y = map_element.size['height'] / 2

# URL에서 중심 좌표와 줌 레벨 추출
url = driver.current_url
center_lat = float(url[url.find("ms=")+3:url.find(",")])
center_lon = float(url[url.find(",")+1:url.find(",16")])
zoom_level = int(url[url.find(",16")+1:url.find("&a")])

# UTM 변환기 설정
utm_proj = Proj(proj='utm', zone=52, ellps='WGS84')
wgs84_proj = Proj(proj='latlong', datum='WGS84')
transformer = Transformer.from_proj(wgs84_proj, utm_proj)

# 중심 좌표를 UTM으로 변환
center_utm_x, center_utm_y = transformer.transform(center_lon, center_lat)

meters_per_pixel = 100 / 61

In [ ]:
import folium

# folium 지도 생성
map_center = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# 마커 추가
folium.Marker(
    location=[center_lat, center_lon],
    popup='중심 좌표',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(map_center)

# 지도 출력
map_center


In [ ]:
# 핀 요소들 찾기
pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")

# 전체 매물 수 계산
total_materials = 0
for pin in pin_elements:
    numbers = re.findall(r'\d+', pin.text)
    if numbers:
        total_materials += int(numbers[0])

print(f"총 매물 수: {total_materials}, 핀 갯수: {len(pin_elements)}")

In [ ]:
# 핀 클릭 및 데이터 수집
for i, pin in enumerate(pin_elements):
    # 핀 위치 계산
    target_x = float(pin.get_attribute("style").split(" ")[-1].replace("px;", ""))
    target_y = float(pin.get_attribute("style").split(" ")[-3].replace("px;", ""))
    
    # 핀의 UTM 좌표 계산
    delta_x = (target_x - map_center_x) * meters_per_pixel
    delta_y = (map_center_y - target_y) * meters_per_pixel  # y축 방향 반전
    
    pin_utm_x = center_utm_x + delta_x
    pin_utm_y = center_utm_y + delta_y
    
    # UTM을 위경도로 변환
    pin_lon, pin_lat = transformer.transform(pin_utm_x, pin_utm_y, direction='INVERSE')

    flag = True
    while flag:    
        try:
            # 핀 클릭
            pin.click()
        except:
            flag = False
            print("클릭할 수 없는 위치")

        print(f"{i+1}번째 핀 클릭")

        time.sleep(2)

        #### 핀에 적혀있는 수와 매물의 수를 비교할 때 사용 
        text = pin.find_element(By.CSS_SELECTOR, 'div.map_cluster_inner>span.sale_number').text
        count = int(re.search(r'\d+', text).group())
        
        try:
            body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
            while True:
                body.send_keys(Keys.END)

                time.sleep(1)
                loader = driver.find_element(By.CSS_SELECTOR, 'div.loader')
        except:
            pass

        items = driver.find_elements(By.CSS_SELECTOR, "div.item")
        
        if count == len(items):
            flag = False

    
    for j, item in enumerate(items):
        item.click()
        driver.implicitly_wait(3)

        ty = driver.find_elements(By.CSS_SELECTOR, "h4.info_title")
        title = ty[1].text

        pay = driver.find_element(By.CSS_SELECTOR, "div.info_article_price ")
        price = pay.text

        table_data = driver.find_elements(By.CSS_SELECTOR, "table.info_table_wrap>tbody")
        ths = table_data[0].find_elements(By.CSS_SELECTOR, "th.table_th")
        tds = table_data[0].find_elements(By.CSS_SELECTOR, "td.table_td")

        # 데이터를 저장할 딕셔너리 초기화
        row_data = {
            '제목': title,
            '월세': price,
            '위도': pin_lat,
            '경도': pin_lon
        }

        # 테이블에서 데이터 추출
        for k, (th, td) in enumerate(zip(ths, tds)):
            header = th.text
            if header in columns:  # columns에 있는 항목만 저장
                row_data[header] = td.text

        # 데이터프레임에 새로운 행 추가
        df.loc[len(df)] = row_data


    close_btns = driver.find_elements(By.CSS_SELECTOR, "button.btn_close")

    for btn in close_btns:
        label = btn.get_attribute("aria-label")
        if label == "상세페이지 닫기":
            btn.click()
            break  # 클릭한 후 반복 종료


In [ ]:
df.head()

In [ ]:
# 혹시 모르니 위도/경도 컬럼 타입을 float로 변환
df['위도'] = df['위도'].astype(float)
df['경도'] = df['경도'].astype(float)

# 지도 초기화
map_with_points = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# 중심 좌표 마커 추가
folium.Marker(
    location=[center_lat, center_lon],
    popup='중심 좌표 (URL)',
    icon=folium.Icon(color='red')
).add_to(map_with_points)

# df 내 매물 좌표 마커 추가
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['제목'],
        icon=folium.Icon(color='blue')
    ).add_to(map_with_points)

map_with_points

In [ ]:
df.to_csv(f"./data/03_부동산/_부동산.csv", encoding='utf-8-sig', index=False)